# Task 05 • Descriptive Statistics & LLM QA (SU Women’s Lacrosse 2024) -- part 2

In [1]:
def extract_text_from_pdfs():
    # Loading Documents:
    loader = DirectoryLoader("dataset_pdf/", glob="**/*.pdf")
    documents = loader.load()

    print(documents)



In [2]:
pip install "unstructured[pdf]"

  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached python_oxmsg-0.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached onnx-1.18.0-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached pikepdf-9.10.2-cp312-cp312-win_amd64.whl.metadata (8.2 kB)
  Using cached google_cloud_vision-3.10.2-py3-none-any.whl.metadata (9.6 kB)
  Using cached effdet-0.4.1-py3-none-any.whl.metadata (33 kB)
  Using cached unstructured_inference-1.0.5-py3-none-any.whl.metadata (5.3 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached timm-1.0.19-py3-none-any.whl.metadata (60 kB)
  Using cached accelerate-1.10.0-py3-none-any.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached torchvision-0.23.0-cp3

error: uninstall-no-record-file

× Cannot uninstall numpy None
╰─> The package's contents are unknown: no RECORD file was found for numpy.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps numpy==1.26.4


In [3]:
pip uninstall onnxruntime

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymupdf pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import fitz  # PyMuPDF
import pandas as pd

# Load the PDF
pdf_path = "dataset_pdf/2024SUStats.pdf"  # Replace with your path if different
doc = fitz.open(pdf_path)

# Extract all text
full_text = ""
for page in doc:
    full_text += page.get_text()

# Extract just the section between "Date" and "TEAM STATISTICS"
start_idx = full_text.find("Date")
end_idx = full_text.find("TEAM STATISTICS")
games_text = full_text[start_idx:end_idx]

# Split and clean lines
lines = [line.strip() for line in games_text.splitlines() if line.strip()]
lines_cleaned = lines[4:]  # Skip headers: "Date", "Opponent", "Score", "Att."
normalized_lines = [line.lstrip("*").strip() for line in lines_cleaned if line != "*"]

# Group every 5 lines into a match record
records = []
i = 0
while i + 4 < len(normalized_lines):
    try:
        date = normalized_lines[i]
        opponent = normalized_lines[i + 1]
        result = normalized_lines[i + 2]
        score = normalized_lines[i + 3]
        attendance = int(normalized_lines[i + 4])

        if '-' in score:
            su_score, opponent_score = map(int, score.split('-'))
        else:
            i += 1
            continue

        records.append([date, opponent, result, su_score, opponent_score, attendance])
        i += 5
    except Exception:
        i += 1  # Skip broken entries

# Create DataFrame and save to CSV
df = pd.DataFrame(records, columns=["Date", "Opponent", "Result", "SU_Score", "Opponent_Score", "Attendance"])
df.to_csv("SU_WLAX_2024_Results_AutoParsed.csv", index=False)

print("Saved to SU_WLAX_2024_Results_AutoParsed.csv")
print(df.head())


Saved to SU_WLAX_2024_Results_AutoParsed.csv
     Date            Opponent Result  SU_Score  Opponent_Score  Attendance
0  Feb.10  at #1 Northwestern      L        15              18         933
1  Feb 14            #18 ARMY      W        18               7        1077
2  Feb 17         #9 MARYLAND  L o t         8               9        2097
3  Feb 24    at #2 Notre Dame      W        16              14         482
4  Mar 02                DUKE      W        15               8        1976


In [15]:
import fitz  # PyMuPDF
import pandas as pd
import re

# ---------- PDF ----------
PDF_PATH = "dataset_pdf/2024SUStats.pdf"   # change if needed
doc = fitz.open(PDF_PATH)
full_text = "".join(p.get_text() for p in doc)

# ---------- TEAM RECORD ----------
try:
    part = full_text.split("RECORD:", 1)[1].split("Date", 1)[0].strip().splitlines()
    data = [x.strip() for x in part if x.strip()]
    df_record = pd.DataFrame({
        "Category": ["All Games", "Conference", "Non-Conference"],
        "Overall": [data[5], data[9], data[13]],
        "Home":    [data[6], data[10], data[14]],
        "Away":    [data[7], data[11], data[15]],
        "Neutral": [data[8], data[12], data[16]],
    })
    df_record.to_csv("SU_Team_Record.csv", index=False)
except Exception as e:
    print("[WARN] Team record not parsed:", e)

# ---------- GAME RESULTS ----------
try:
    games_part = full_text.split("Date", 1)[1].split("TEAM STATISTICS", 1)[0]
    lines = [ln.strip().lstrip("*") for ln in games_part.splitlines() if ln.strip() and ln != "*"]
    lines = lines[4:]  # skip headers

    games, i = [], 0
    while i + 4 < len(lines):
        try:
            date, opponent, result, score, attend = lines[i:i+5]
            nums = list(map(int, re.findall(r"\d+", score)))
            su_sc, op_sc = (nums + [0, 0])[:2]
            attend = int(re.findall(r"\d+", attend)[0])
            games.append([date, opponent, result, su_sc, op_sc, attend])
            i += 5
        except Exception:
            i += 1

    pd.DataFrame(
        games,
        columns=["Date", "Opponent", "Result", "SU_Score", "Opponent_Score", "Attendance"]
    ).to_csv("SU_Game_Results_2024.csv", index=False)
except Exception as e:
    print("[WARN] Game results not parsed:", e)

# ---------- PERIOD TABLES (simple + robust) ----------
def _find_total_idx(nums):
    for i in range(5, min(len(nums), 8)):          # 1st–4th + optional OT/OT2
        if sum(nums[:i]) == nums[i]:
            return i
    return None

def _labels(n):
    base = ["1st", "2nd", "3rd", "4th"]
    if n >= 5: base.append("OT")
    if n >= 6: base.append("OT2")
    base.append("Total")
    return base

def _normalize(nums, want):
    if not nums:
        return [0]*want + [0]
    ti = _find_total_idx(nums)
    if ti is None:
        base = nums[:want]
        return base + [sum(base)]
    arr = nums[:ti+1]                 # include Total
    periods = len(arr) - 1
    if periods < want:                # pad before Total
        arr = arr[:-1] + [0]*(want - periods) + arr[-1:]
    elif periods > want:              # trim before Total
        arr = arr[:want] + [arr[-1]]
    arr[-1] = sum(arr[:-1])           # ensure Total is correct
    return arr

def _block_after(label):
    m = re.search(rf"{re.escape(label)}\s*(.+?)(?:\n\s*\n|$)", full_text, flags=re.DOTALL|re.IGNORECASE)
    return m.group(1) if m else None

def extract_stats_block(label, alt=None):
    blk = _block_after(label) or (_block_after(alt) if alt else None)
    if blk is None:
        raise ValueError(f"Block not found: {label}")

    def team_nums(team):
        m = re.search(rf"{team}.*", blk, flags=re.IGNORECASE)
        return list(map(int, re.findall(r"\d+", m.group(0)))) if m else None

    su = team_nums("Syracuse")
    opp = team_nums("Opponents")

    if su is None or opp is None:
        all_nums = list(map(int, re.findall(r"\d+", blk)))
        ti = _find_total_idx(all_nums) or (len(all_nums)//2 - 1)
        su = all_nums[:ti+1]
        opp = all_nums[ti+1:]

    want = max(4, min(6, len(su)-1), min(6, len(opp)-1))  # clamp 4..6
    labels = _labels(want)
    su = _normalize(su, want)
    opp = _normalize(opp, want)

    # final safety: align lengths
    m = min(len(labels), len(su), len(opp))
    return pd.DataFrame({"Period": labels[:m], "Syracuse": su[:m], "Opponents": opp[:m]})

# Parse and save
df_goals = extract_stats_block("Goals by Period")
df_saves = extract_stats_block("Saves by Period")
df_shots = extract_stats_block("Shots by Period")
df_sog   = extract_stats_block("Shots on Goal", alt="Shots on Goal by Period")

df_goals.to_csv("SU_Goals_By_Period_2024.csv", index=False)
df_saves.to_csv("SU_Saves_By_Period_2024.csv", index=False)
df_shots.to_csv("SU_Shots_By_Period_2024.csv", index=False)
df_sog.to_csv("SU_SOG_By_Period_2024.csv", index=False)

print("CSV files written.")


CSV files written.


In [16]:
import pandas as pd

# ✅ Make sure this line exists:
games = pd.read_csv("SU_Game_Results_2024.csv")

# ✅ Now this will work correctly
total_games = len(games)

# Win/Loss ratio
wins = len(games[games["Result"].str.startswith("W")])
losses = len(games[games["Result"].str.startswith("L")])
win_rate = wins / total_games

# Average score
avg_su_score = games["SU_Score"].mean()
avg_opp_score = games["Opponent_Score"].mean()

# Attendance insights
most_attended = games.sort_values("Attendance", ascending=False).head(3)

# Print summary
print(f"Games played: {total_games}, Wins: {wins}, Win %: {win_rate:.2f}")
print(f"Average SU Score: {avg_su_score:.2f}, Opponent Score: {avg_opp_score:.2f}")
print("Most attended games:\n", most_attended)



Games played: 21, Wins: 0, Win %: 0.00
Average SU Score: 575.14, Opponent Score: 0.00
Most attended games:
            Date Opponent          Result  SU_Score  Opponent_Score  Attendance
2   #9 MARYLAND    L o t             8-9      2097               0          24
10         1238   Mar 30      LOUISVILLE         0               0          22
16          268   Apr 26  vs #6 Virginia         0               0          19


In [17]:
import pandas as pd
import os

# Load relevant CSV files
csv_files = {
    "game_results": "SU_Game_Results_2024.csv",
    "goals_by_period": "SU_Goals_By_Period_2024.csv",
    "saves_by_period": "SU_Saves_By_Period_2024.csv",
    "shots_by_period": "SU_Shots_By_Period_2024.csv",
    "sog_by_period": "SU_SOG_By_Period.csv",
    "team_record": "SU_Team_Record.csv"
}

# Load CSVs into dictionary
dataframes = {}
for key, file in csv_files.items():
    if os.path.exists(file):
        dataframes[key] = pd.read_csv(file)
    else:
        dataframes[key] = pd.DataFrame()

# Create stats summary dictionary
stats_summary = {}

# Team record stats
df_team = dataframes["team_record"]
if not df_team.empty:
    stats_summary["Total Games"] = df_team['Total Games'].sum() if 'Total Games' in df_team.columns else len(df_team)
    stats_summary["Wins"] = df_team['Wins'].sum() if 'Wins' in df_team.columns else None
    stats_summary["Losses"] = df_team['Losses'].sum() if 'Losses' in df_team.columns else None

# Goal statistics
df_goals = dataframes["goals_by_period"]
if not df_goals.empty:
    total_goals = df_goals.select_dtypes(include='number').sum().sum()
    stats_summary["Total Goals"] = total_goals
    stats_summary["Average Goals per Game"] = total_goals / len(df_goals)

# Save statistics
df_saves = dataframes["saves_by_period"]
if not df_saves.empty:
    total_saves = df_saves.select_dtypes(include='number').sum().sum()
    stats_summary["Total Saves"] = total_saves

# Shot statistics
df_shots = dataframes["shots_by_period"]
if not df_shots.empty:
    total_shots = df_shots.select_dtypes(include='number').sum().sum()
    stats_summary["Total Shots"] = total_shots
    if "Total Goals" in stats_summary:
        stats_summary["Shot Conversion Rate"] = stats_summary["Total Goals"] / total_shots

# Save percentage
if "Total Saves" in stats_summary and "Total Shots" in stats_summary:
    stats_summary["Save Percentage"] = stats_summary["Total Saves"] / stats_summary["Total Shots"]

# Convert to DataFrame and display
stats_df = pd.DataFrame(list(stats_summary.items()), columns=["Metric", "Value"])
display(stats_df)


C:\Users\manal\AppData\Local\Temp\ipykernel_19596\1031439173.py:51: RuntimeWarning: invalid value encountered in scalar divide
  stats_summary["Shot Conversion Rate"] = stats_summary["Total Goals"] / total_shots
C:\Users\manal\AppData\Local\Temp\ipykernel_19596\1031439173.py:55: RuntimeWarning: invalid value encountered in scalar divide
  stats_summary["Save Percentage"] = stats_summary["Total Saves"] / stats_summary["Total Shots"]


,Metric,Value
0,Total Games,3.0
1,Wins,NaN
2,Losses,NaN
3,Total Goals,0.0
4,Average Goals per Game,0.0
5,Total Saves,0.0
6,Total Shots,0.0
7,Shot Conversion Rate,NaN
8,Save Percentage,NaN


## LLM Evaluation Prompts (2024)

Copy/paste into your LLM of choice (ChatGPT / Claude / Copilot). Provide the CSVs produced by this notebook as context.

**Prompt A — Baseline sanity checks**
- How many games did Syracuse play in 2024?
- What was the overall record? Home/Away/Neutral splits?
- Average goals scored vs allowed per game?

**Prompt B — Deeper reasoning**
- If the coach wants to win two more games next season, should the emphasis be offense or defense? Defend your answer with 2024 metrics (e.g., goals/shot quality, saves, shot clock violations, clears, turnovers).
- Identify one player (if player-level data is available) or one phase (e.g., 1st vs 4th period) to prioritize. Explain the potential impact with evidence from 2024 stats.

**Prompt C — Visualization requests**
- Create a bar chart comparing goals, shots, and saves by period.
- Plot goals scored vs allowed by game to spot trends or outliers.

> After the LLM answers, validate each claim against the computed stats from this notebook. Record failures and fixes in your README.
